In [21]:
!apt-get install xvfb

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [22]:
!xvfb-run python your_script.py

'xvfb-run' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
%pip install cryptography


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
%pip install pycryptodome

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes
import math
from collections import Counter
from scipy.stats import chisquare
from scipy.spatial.distance import hamming

def encrypt_file(file_path, key, iv):
    try:
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        encryptor = cipher.encryptor()

        padder = padding.PKCS7(128).padder()

        with open(file_path, 'rb') as file:
            data = file.read()

        padded_data = padder.update(data) + padder.finalize()
        encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

        return data, padded_data, encrypted_data
    except Exception as e:
        print("Error during encryption:", str(e))
        return None, None, None

def analyze_frequency(data_bytes):
    return Counter(data_bytes)

def calculate_entropy(data_bytes):
    frequency = analyze_frequency(data_bytes)
    total = sum(frequency.values())
    entropy = -sum((count / total) * math.log2(count / total) for count in frequency.values())
    return entropy

def chi_square_test(data_bytes):
    frequency = analyze_frequency(data_bytes)
    observed = list(frequency.values())
    expected = [sum(observed) / len(frequency)] * len(frequency)
    chi_stat, p_value = chisquare(f_obs=observed, f_exp=expected)
    return chi_stat, p_value

def hamming_distance(binary1, binary2):
    min_length = min(len(binary1), len(binary2))
    distance = sum(b1 != b2 for b1, b2 in zip(binary1[:min_length], binary2[:min_length]))
    return distance, min_length

def analyze_ciphertext(data):
    frequency = analyze_frequency(data)
    total_bits = len(data) * 8
    ones_count = sum(bin(byte).count('1') for byte in data)
    zeros_count = total_bits - ones_count

    entropy = calculate_entropy(data)
    chi_stat, p_value = chi_square_test(data)

    return {
        "Frequency Analysis": dict(frequency),
        "Total Bits": total_bits,
        "1's Percentage": f"{(ones_count / total_bits) * 100:.2f}%",
        "0's Percentage": f"{(zeros_count / total_bits) * 100:.2f}%",
        "Shannon Entropy": f"{entropy:.4f}",
        "Chi-Square Statistic": f"{chi_stat:.4f}",
        "P-Value": f"{p_value:.4e}"
    }

def process_file():
    file_path = input("Enter the full path of the file to encrypt: ").strip()
    if not os.path.exists(file_path):
        print("File not found. Please provide a valid file path.")
        return

    binary_key = input("Enter a 256-bit binary key (256 bits = 32 bytes): ").strip()
    if len(binary_key) != 256 or not all(bit in '01' for bit in binary_key):
        print("Invalid key. Please provide exactly 256 bits (0s and 1s).")
        return

    key = int(binary_key, 2).to_bytes(32, byteorder='big')
    iv = os.urandom(16)

    plaintext, padded_plaintext, ciphertext = encrypt_file(file_path, key, iv)
    if plaintext is None:
        print("Encryption failed.")
        return

    # Analyze plaintext and ciphertext
    plaintext_binary = ''.join(format(byte, '08b') for byte in plaintext)
    ciphertext_binary = ''.join(format(byte, '08b') for byte in ciphertext)

    result_plaintext = analyze_ciphertext(plaintext)
    result_ciphertext = analyze_ciphertext(ciphertext)

    print("\nPlaintext Analysis:")
    for key, value in result_plaintext.items():
        print(f"{key}: {value}")

    print("\nCiphertext Analysis:")
    for key, value in result_ciphertext.items():
        print(f"{key}: {value}")

    hamming_dist, compared_length = hamming_distance(plaintext_binary, ciphertext_binary)
    hamming_percentage = (hamming_dist / compared_length) * 100

    print(f"\nHamming Distance: {hamming_dist}")
    print(f"Compared Length: {compared_length}")
    print(f"Hamming Distance Percentage: {hamming_percentage:.2f}%")

    # Re-encrypt the ciphertext using a 128-bit random key
    random_key = get_random_bytes(16)
    cipher_ecb = AES.new(random_key, AES.MODE_ECB)
    re_encrypted = cipher_ecb.encrypt(pad(ciphertext, AES.block_size))

    # Analyze re-encrypted data
    re_encrypted_binary = ''.join(format(byte, '08b') for byte in re_encrypted)
    re_encrypted_entropy = calculate_entropy(re_encrypted)
    hamming_dist_re, compared_length_re = hamming_distance(ciphertext_binary, re_encrypted_binary)
    hamming_percentage_re = (hamming_dist_re / compared_length_re) * 100

    print("\nRe-encrypted Shannon Entropy:", f"{re_encrypted_entropy:.4f}")
    print("Hamming Distance (Re-encrypted):", hamming_dist_re)
    print("Compared Length (Re-encrypted):", compared_length_re)
    print("Hamming Distance Percentage (Re-encrypted):", f"{hamming_percentage_re:.2f}%")

if __name__ == "__main__":
    try:
        process_file()
    except KeyboardInterrupt:
        print("\nProcess interrupted by user.")




Plaintext Analysis:
Frequency Analysis: {37: 2159, 80: 2417, 68: 2683, 70: 3128, 45: 2319, 49: 4635, 46: 3960, 52: 3969, 10: 5842, 208: 2162, 212: 2150, 197: 2197, 216: 2171, 53: 4531, 32: 13600, 48: 11400, 111: 6067, 98: 4019, 106: 3216, 60: 3185, 47: 12385, 83: 2602, 71: 2167, 84: 2556, 40: 2304, 99: 5025, 104: 3468, 97: 6060, 112: 3461, 116: 6382, 101: 10449, 114: 6164, 41: 2468, 62: 3117, 110: 6178, 100: 4768, 56: 3672, 79: 2626, 117: 4442, 72: 2331, 115: 5426, 65: 2584, 108: 5517, 103: 4295, 105: 6525, 109: 3704, 102: 3295, 121: 2761, 92: 2062, 87: 2352, 44: 2190, 118: 2742, 107: 2446, 57: 3341, 50: 4192, 73: 2412, 51: 4099, 54: 3953, 55: 3870, 122: 2247, 67: 2634, 66: 2428, 91: 2537, 82: 3116, 93: 2653, 76: 2560, 120: 2751, 218: 2221, 133: 2189, 75: 2288, 143: 2122, 219: 2320, 16: 2274, 190: 2319, 239: 2233, 175: 2235, 173: 2234, 171: 2140, 34: 2218, 69: 2556, 234: 2276, 113: 2715, 220: 2190, 4: 2176, 13: 2127, 210: 2076, 94: 2098, 154: 2157, 162: 2173, 6: 2146, 11: 2169, 148: 2

Large file encryption

In [11]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes
import math
from collections import Counter
from scipy.stats import chisquare

def encrypt_file(file_path, key, iv):
    try:
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        encryptor = cipher.encryptor()

        padder = padding.PKCS7(128).padder()

        with open(file_path, 'rb') as file:
            data = file.read()

        padded_data = padder.update(data) + padder.finalize()
        encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

        return data, padded_data, encrypted_data
    except Exception as e:
        print("Error during encryption:", str(e))
        return None, None, None

def analyze_frequency(data_bytes):
    return Counter(data_bytes)

def calculate_entropy(data_bytes):
    frequency = analyze_frequency(data_bytes)
    total = sum(frequency.values())
    entropy = -sum((count / total) * math.log2(count / total) for count in frequency.values())
    return entropy

def chi_square_test(data_bytes):
    frequency = analyze_frequency(data_bytes)
    observed = list(frequency.values())
    expected = [sum(observed) / len(frequency)] * len(frequency)
    chi_stat, p_value = chisquare(f_obs=observed, f_exp=expected)
    return chi_stat, p_value

def analyze_ciphertext(data):
    frequency = analyze_frequency(data)
    total_bits = len(data) * 8
    ones_count = sum(bin(byte).count('1') for byte in data)
    zeros_count = total_bits - ones_count

    entropy = calculate_entropy(data)
    chi_stat, p_value = chi_square_test(data)

    return {
        "Frequency Analysis": dict(frequency),
        "Total Bits": total_bits,
        "1's Percentage": f"{(ones_count / total_bits) * 100:.2f}%",
        "0's Percentage": f"{(zeros_count / total_bits) * 100:.2f}%",
        "Shannon Entropy": f"{entropy:.4f}",
        "Chi-Square Statistic": f"{chi_stat:.4f}",
        "P-Value": f"{p_value:.4e}"
    }

def hamming_distance_stream(data1, data2):
    min_length = min(len(data1), len(data2))
    distance = sum(bin(b1 ^ b2).count('1') for b1, b2 in zip(data1[:min_length], data2[:min_length]))
    return distance, min_length

def process_file():
    file_path = input("Enter the full path of the file to encrypt: ").strip()
    if not os.path.exists(file_path):
        print("File not found. Please provide a valid file path.")
        return

    binary_key = input("Enter a 256-bit binary key (256 bits = 32 bytes): ").strip()
    if len(binary_key) != 256 or not all(bit in '01' for bit in binary_key):
        print("Invalid key. Please provide exactly 256 bits (0s and 1s).")
        return

    key = int(binary_key, 2).to_bytes(32, byteorder='big')
    iv = os.urandom(16)

    plaintext, padded_plaintext, ciphertext = encrypt_file(file_path, key, iv)
    if plaintext is None:
        print("Encryption failed.")
        return

    # Analyze plaintext and ciphertext
    result_plaintext = analyze_ciphertext(plaintext)
    result_ciphertext = analyze_ciphertext(ciphertext)

    print("\nPlaintext Analysis:")
    for key, value in result_plaintext.items():
        print(f"{key}: {value}")

    print("\nCiphertext Analysis:")
    for key, value in result_ciphertext.items():
        print(f"{key}: {value}")

    # Calculate Hamming distance between plaintext and ciphertext
    hamming_dist, compared_length = hamming_distance_stream(plaintext, ciphertext)
    hamming_percentage = (hamming_dist / compared_length) * 100

    print(f"\nHamming Distance: {hamming_dist}")
    print(f"Compared Length: {compared_length}")
    print(f"Hamming Distance Percentage: {hamming_percentage:.2f}%")

    # Re-encrypt the ciphertext using a 128-bit random key
    random_key = get_random_bytes(16)
    cipher_ecb = AES.new(random_key, AES.MODE_ECB)
    re_encrypted = cipher_ecb.encrypt(pad(ciphertext, AES.block_size))

    # Analyze re-encrypted ciphertext
    result_re_encrypted = analyze_ciphertext(re_encrypted)
    print("\nRe-encrypted Ciphertext Analysis:")
    for key, value in result_re_encrypted.items():
        print(f"{key}: {value}")

    # Hamming distance between original and re-encrypted ciphertext
    hamming_dist_re, compared_length_re = hamming_distance_stream(ciphertext, re_encrypted)
    hamming_percentage_re = (hamming_dist_re / compared_length_re) * 100

    print(f"\nRe-encrypted Hamming Distance: {hamming_dist_re}")
    print(f"Compared Length (Re-encrypted): {compared_length_re}")
    print(f"Re-encrypted Hamming Distance Percentage: {hamming_percentage_re:.2f}%")

if __name__ == "__main__":
    try:
        process_file()
    except KeyboardInterrupt:
        print("\nProcess interrupted by user.")



Plaintext Analysis:
Frequency Analysis: {0: 2510012, 32: 486605, 102: 479166, 116: 476050, 121: 481422, 112: 479868, 105: 480457, 115: 479669, 111: 482193, 109: 484211, 2: 533838, 50: 487844, 97: 477663, 118: 475954, 99: 476743, 49: 478655, 52: 493938, 8: 487661, 114: 483346, 101: 484359, 24: 484065, 110: 480389, 107: 485522, 26: 486219, 100: 495603, 245: 486933, 6: 484503, 5: 475046, 255: 285288843, 241: 483382, 220: 477055, 69: 486525, 233: 478864, 189: 475134, 230: 476665, 217: 481607, 72: 489211, 183: 482610, 150: 482840, 44: 489572, 216: 484486, 35: 484540, 238: 471936, 239: 475774, 120: 475348, 54: 483298, 45: 482225, 51: 476760, 48: 484294, 57: 485224, 56: 490007, 46: 475681, 47: 472761, 77: 482258, 80: 479052, 71: 473196, 65: 515037, 86: 483469, 67: 478217, 108: 485436, 104: 478044, 58: 487269, 119: 472204, 103: 477694, 98: 479790, 61: 476704, 117: 486864, 95: 475666, 113: 482785, 122: 481289, 53: 487738, 128: 543555, 28: 489275, 31: 479702, 172: 486884, 64: 491123, 187: 47488